In [17]:
import pandas as pd
import numpy as np

--- Day 2: Dive! ---

Now, you need to figure out how to pilot this thing.

It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

forward X increases the horizontal position by X units.
down X increases the depth by X units.
up X decreases the depth by X units.
Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.

The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:

forward 5
down 5
forward 8
up 3
down 8
forward 2
Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

forward 5 adds 5 to your horizontal position, a total of 5.
down 5 adds 5 to your depth, resulting in a value of 5.
forward 8 adds 8 to your horizontal position, a total of 13.
up 3 decreases your depth by 3, resulting in a value of 2.
down 8 adds 8 to your depth, resulting in a value of 10.
forward 2 adds 2 to your horizontal position, a total of 15.
After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [2]:
df = pd.read_csv('input.txt', header=None, sep="\s+", names=["direction","magnitude"])

In [3]:
df.head()

,direction,magnitude
0,forward,8
1,down,9
2,up,1
3,forward,2
4,down,6


In [5]:
translate_directions = {"forward": 1, "down": 1, "up": -1}

In [4]:
set(df.direction)

{'down', 'forward', 'up'}

In [6]:
df["vector"] = df.apply(lambda x: translate_directions[x["direction"]]*x["magnitude"], axis=1)

In [7]:
df.head()

,direction,magnitude,vector
0,forward,8,8
1,down,9,9
2,up,1,-1
3,forward,2,2
4,down,6,6


In [8]:
df[df.direction=="forward"]["vector"].sum()

1911

In [9]:
df[df.direction!="forward"]["vector"].sum()

779

In [10]:
df[df.direction=="forward"]["vector"].sum()*df[df.direction!="forward"]["vector"].sum()

1488669

In [11]:
1911*779

1488669

--- Part Two ---

Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.

In addition to horizontal position and depth, you'll also need to track a third value, aim, which also starts at 0. The commands also mean something entirely different than you first thought:

down X increases your aim by X units.
up X decreases your aim by X units.
forward X does two things:
It increases your horizontal position by X units.
It increases your depth by your aim multiplied by X.
Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.

Now, the above example does something different:

forward 5 adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.
down 5 adds 5 to your aim, resulting in a value of 5.
forward 8 adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.
up 3 decreases your aim by 3, resulting in a value of 2.
down 8 adds 8 to your aim, resulting in a value of 10.
forward 2 adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.
After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [14]:
df.loc[df.direction=="forward"]["vector"].cumsum()

0         8
3        10
5        16
9        19
10       27
       ... 
991    1896
992    1898
996    1902
998    1909
999    1911
Name: vector, Length: 400, dtype: int64

In [15]:
df.loc[df.direction=="forward"]["vector"]

0      8
3      2
5      6
9      3
10     8
      ..
991    9
992    2
996    4
998    7
999    2
Name: vector, Length: 400, dtype: int64

In [18]:
df["horizontal"] = np.zeros_like(len(df))

In [19]:
df.loc[df.direction=="forward", "horizontal"] = df.loc[df.direction=="forward", "vector"].cumsum()

In [20]:
df.head()

,direction,magnitude,vector,horizontal
0,forward,8,8,8
1,down,9,9,0
2,up,1,-1,0
3,forward,2,2,10
4,down,6,6,0


In [23]:
df["horizontal"] = df.horizontal.replace(to_replace=0, method="ffill")

In [24]:
df.head()

,direction,magnitude,vector,horizontal
0,forward,8,8,8
1,down,9,9,8
2,up,1,-1,8
3,forward,2,2,10
4,down,6,6,10


In [25]:
df["depth"] = np.zeros_like(len(df))

In [26]:
df.loc[df.direction!="forward", "depth"] = df.loc[df.direction!="forward", "vector"].cumsum()

In [27]:
df.head()

,direction,magnitude,vector,horizontal,depth
0,forward,8,8,8,0
1,down,9,9,8,9
2,up,1,-1,8,8
3,forward,2,2,10,0
4,down,6,6,10,14


In [29]:
df["depth"] = df.depth.replace(to_replace=0, method="ffill")

In [30]:
df.head()

,direction,magnitude,vector,horizontal,depth
0,forward,8,8,8,0
1,down,9,9,8,9
2,up,1,-1,8,8
3,forward,2,2,10,8
4,down,6,6,10,14


In [37]:
df["aim"] = df.depth

In [38]:
df.head()

,direction,magnitude,vector,horizontal,depth,aim
0,forward,8,8,8,0,0
1,down,9,9,8,9,9
2,up,1,-1,8,8,8
3,forward,2,2,10,8,8
4,down,6,6,10,14,14


In [63]:
df["new_depth_vector"] = np.zeros_like(len(df))
#del df["new_vector"]

In [50]:
df.head()

,direction,magnitude,vector,horizontal,depth,aim,new_depth
0,forward,8,8,8,0,0,0
1,down,9,9,8,9,9,0
2,up,1,-1,8,8,8,0
3,forward,2,2,10,8,8,0
4,down,6,6,10,14,14,0


In [64]:
mask = df.direction=="forward"
df.loc[mask,"new_depth_vector"] = df.loc[mask,"aim"]*df.loc[mask,"magnitude"]

In [65]:
df.head()

,direction,magnitude,vector,horizontal,depth,aim,new_depth,new_depth_vector
0,forward,8,8,8,0,0,0,0
1,down,9,9,8,9,9,9,0
2,up,1,-1,8,8,8,-1,0
3,forward,2,2,10,8,8,16,16
4,down,6,6,10,14,14,6,0


In [66]:
mask = df.direction!="forward"
df.loc[mask,"new_depth_vector"] = df.loc[mask,"vector"]

In [67]:
df.head(10)

,direction,magnitude,vector,horizontal,depth,aim,new_depth,new_depth_vector
0,forward,8,8,8,0,0,0,0
1,down,9,9,8,9,9,9,9
2,up,1,-1,8,8,8,-1,-1
3,forward,2,2,10,8,8,16,16
4,down,6,6,10,14,14,6,6
5,forward,6,6,16,14,14,84,84
6,down,5,5,16,19,19,5,5
7,down,4,4,16,23,23,4,4
8,down,2,2,16,25,25,2,2
9,forward,3,3,19,25,25,75,75


In [68]:
df.loc[df.direction=="forward"]

,direction,magnitude,vector,horizontal,depth,aim,new_depth,new_depth_vector
0,forward,8,8,8,0,0,0,0
3,forward,2,2,10,8,8,16,16
5,forward,6,6,16,14,14,84,84
9,forward,3,3,19,25,25,75,75
10,forward,8,8,27,25,25,200,200
...,...,...,...,...,...,...,...,...
991,forward,9,9,1896,763,763,6867,6867
992,forward,2,2,1898,763,763,1526,1526
996,forward,4,4,1902,773,773,3092,3092
998,forward,7,7,1909,779,779,5453,5453


In [74]:
df["new_depth"] = df["new_depth_vector"].cumsum()

In [75]:
df.head()

,direction,magnitude,vector,horizontal,depth,aim,new_depth,new_depth_vector
0,forward,8,8,8,0,0,0,0
1,down,9,9,8,9,9,9,9
2,up,1,-1,8,8,8,8,-1
3,forward,2,2,10,8,8,24,16
4,down,6,6,10,14,14,30,6


In [76]:
df[df.new_depth==0]

,direction,magnitude,vector,horizontal,depth,aim,new_depth,new_depth_vector
0,forward,8,8,8,0,0,0,0


In [77]:
df.tail()

,direction,magnitude,vector,horizontal,depth,aim,new_depth,new_depth_vector
995,down,8,8,1898,773,773,606324,8
996,forward,4,4,1902,773,773,609416,3092
997,down,6,6,1902,779,779,609422,6
998,forward,7,7,1909,779,779,614875,5453
999,forward,2,2,1911,779,779,616433,1558


In [79]:
df.tail(1)["horizontal"] * df.tail(1)["new_depth"]

999    1178003463
dtype: int64

In [98]:
def part2(input_file="input.txt"):
    
    df = pd.read_csv(input_file, header=None, sep="\s+", names=["direction","magnitude"])
    
    translate_directions = {"forward": 1, "down": 1, "up": -1}
    df["vector"] = df.apply(lambda x: translate_directions[x["direction"]] * x["magnitude"], axis=1)
    
    horizontal_mask = df.direction == "forward"
    df["horizontal"] = np.zeros_like(len(df))
    df.loc[horizontal_mask, "horizontal"] = df.loc[horizontal_mask, "vector"].cumsum()
    df["horizontal"] = df.horizontal.replace(to_replace=0, method="ffill")
    
    depth_mask = df.direction != "forward"
    df["aim_vector"] = np.zeros_like(len(df))
    df["aim"] = np.zeros_like(len(df))
    df["depth2_vector"] = np.zeros_like(len(df))
    df["depth2"] = np.zeros_like(len(df))
    
    df.loc[depth_mask,"aim_vector"] = df.loc[depth_mask,"vector"]
    df["aim"] = df["aim_vector"].cumsum()
    
    #df.loc[depth_mask, "depth2_vector"] = df.loc[depth_mask, "vector"]
    df.loc[horizontal_mask, "depth2_vector"] = df.loc[horizontal_mask].apply(lambda x: x["magnitude"] * x["aim"], axis=1)
    df["depth2"] = df["depth2_vector"].cumsum()
    
    return df

In [95]:
df = part2()

In [96]:
df.tail()

,direction,magnitude,vector,horizontal,aim_vector,aim,depth2_vector,depth2
995,down,8,8,1898,8,773,8,606324
996,forward,4,4,1902,0,773,3092,609416
997,down,6,6,1902,6,779,6,609422
998,forward,7,7,1909,0,779,5453,614875
999,forward,2,2,1911,0,779,1558,616433


In [97]:
df.tail(1)["horizontal"] * df.tail(1)["depth2"]

999    1178003463
dtype: int64

In [99]:
df = part2()

In [100]:
df.tail()

,direction,magnitude,vector,horizontal,aim_vector,aim,depth2_vector,depth2
995,down,8,8,1898,8,773,0,605551
996,forward,4,4,1902,0,773,3092,608643
997,down,6,6,1902,6,779,0,608643
998,forward,7,7,1909,0,779,5453,614096
999,forward,2,2,1911,0,779,1558,615654


In [102]:
df.tail(1)["horizontal"] * df.tail(1)["depth2"]

999    1176514794
dtype: int64

In [103]:
var_string = "{var} other things in string"

In [105]:
print(var_string)

{var} other things in string


In [106]:
var_string.format(var="some text")

'some text other things in string'

In [107]:
var_dict = {"var": "some text in a dictionary"}

In [108]:
var_string

'{var} other things in string'

In [109]:
var_string.format(**var_dict)

'some text in a dictionary other things in string'

In [110]:
df.head()

,direction,magnitude,vector,horizontal,aim_vector,aim,depth2_vector,depth2
0,forward,8,8,8,0,0,0,0
1,down,9,9,8,9,9,0,0
2,up,1,-1,8,-1,8,0,0
3,forward,2,2,10,0,8,16,16
4,down,6,6,10,6,14,0,16


In [113]:
dir_string = "{direction} from the DataFrame"
for idx, row in df.head().iterrows():
    print(dir_string.format(**row))

forward from the DataFrame
down from the DataFrame
up from the DataFrame
forward from the DataFrame
down from the DataFrame


In [114]:
for idx, row in df.head().iterrows():
    print(var_string.format(**row))

KeyError: 'var'